In [1]:
import json

import wikipedia as wp
import pandas as pd
from tqdm.notebook import tqdm

from marvel import config

In [ ]:
def get_all_links_single_page(page_name):
    return wp.page(page_name).links


def get_all_links_all_pages(char_pages):
    links = []
    for page in char_pages:
        links += get_all_links_single_page(page)
    return links


def get_links_to_other_characters(char_dict, page_dict):
    name = char_dict['name']
    all_links = get_all_links_all_pages(char_dict['pages'])
    links_to_others = {}
    for link in all_links:
        if link in page_dict:
            linked_chars = page_dict.get(link, [])
            for char in linked_chars:
                curr_links = links_to_others.get(char, 0)
                links_to_others[char] = curr_links + 1
    
    links_to_others_json = [{"source": name, "target": target, "count": count} for target, count in links_to_others.items()]
    return links_to_others_json

In [2]:
mcu_char_page_names = json.loads((config.cleandata / 'all_character_pages_processed.json').open().read())

# Converting to mapping from page names -> characters
pages_to_names = {}
for dct in mcu_char_page_names:
    charname = dct['name']
    for page_name in dct['pages']:
        linked_chars = pages_to_names.get(page_name, [])
        linked_chars.append(charname)
        pages_to_names[page_name] = linked_chars


all_links = [get_links_to_other_characters(char_dict, pages_to_names) for char_dict in tqdm(mcu_char_page_names)]

all_links
json.dump(all_links, (config.cleandata / 'all_character_links.json').open('w'))